In [1]:
import os
import numpy as np
from multiprocessing import Pool
from numpy.lib.function_base import append
import scipy as sp
import random  
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
num_threads = 28
feature_size=128
PATH = '/work/angel00540/work/cgh_2020/paper_e1/data/'
IN_PATH = []
TEST_PATH = '/home/angel00540/work_/work/cgh_2020/paper_e1/'
WEIGHT_PATH = '/home/angel00540/work_/work/cgh_2020/paper_e1/weight/InceptionV3_paper_e1_c4_p4_bs128_data_0223_2222.59-0.96-0.07.hdf5'

2022-03-09 20:37:28.535345: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def mfcc_3ch_spectrum(wavfile):
    import librosa
    from skimage.transform import resize

    fix_n_sec = 2
    mini_sec =0.3
    num_channels = 3
    window_sizes = [25, 50, 100]
    hop_sizes = [10, 25, 50]
    y=[]
    try:
        y, sr = librosa.load(wavfile)
    except:
        print("read error: ",wavfile)

    if len(y)>0:
        # pad if len less 1 sec
        if len(y)/sr > fix_n_sec:
            x = y[:fix_n_sec * sr]
            run=1
        elif mini_sec < len(y)/sr < fix_n_sec :
            z = int(fix_n_sec*sr - len(y))
            if(z%2==0):
                z1 = int(z/2)
                z2 = int(z/2)
            else:
                z1 = int((z-1)/2)
                z2 = int(z1 + 1)
            try:
                x = np.pad(y,(z1,z2), 'linear_ramp', end_values=(0, 0))
                run=1
            except:
                print("pad error: ",wavfile)
            # ([0]*append_zero)+y+([0]*append_zero)
        elif len(y)/sr < mini_sec:
            print('len < '+mini_sec+': ',wavfile)
            run=0

        if run==1:
            # audio normalizedy
            normalizedy = librosa.util.normalize(x)

            specs = []
            for i in range(num_channels):

                window_length = int(round(window_sizes[i]*sr/1000))
                # print('win_len: ',window_length)
                hop_length = int(round(hop_sizes[i]*sr/1000))
                mel = librosa.feature.melspectrogram(
                    y=normalizedy, sr=sr,n_fft=window_length,hop_length=hop_length, win_length=window_length)
                mellog = np.log(mel + 1e-9)

                spec = librosa.util.normalize(mellog)

                spec = resize(mellog, (128, feature_size))
                spec = np.asarray(spec)
                specs.append(spec)

            # list to np array
            specs = np.asarray(specs)
            specs = np.moveaxis(specs,0,2)
            print('data.shape : ',specs.shape)
            return specs


In [3]:
classes = []
labels = [ 'Velar', 'Stopping','Affricate','Consonant vowel']

In [4]:
for n, _, _ in os.walk(os.path.abspath(TEST_PATH+'/ori/')):
    classes.append(n.split('/')[-1])

classes = classes[1:]
classes = [int(x) for x in classes]
classes.sort()

classes = [str(x) for x in classes]
print(classes)

['4', '6', '7', '14']


In [5]:
IN_PATH = []
for root2, dirs2, files2 in os.walk(os.path.abspath(PATH)):
    for file2 in files2:
        if((file2[-5:] != '_.wav') & (file2[-4:] != '.npy')):
            IN_PATH.append(os.path.join(root2, file2))
        

In [9]:
wav_npdata = []
for i in range(5):
    index_data = random.randint(0,len(IN_PATH))
    print(f"Data index: {index_data}")
    wav_file = IN_PATH[index_data]
    wav_file_name = wav_file.split("/")[-1]
    print(f"wav_file_name : {wav_file_name}" )
    wav_mfcc = mfcc_3ch_spectrum(wav_file)
    print(f"wav_mfcc.shape : {wav_mfcc.shape}" )
    wav_npdata.append(wav_mfcc)
wav_npdata=np.asarray(wav_npdata)
print(f"\nwav_npdata.shape : {wav_npdata.shape}" )

Data index: 1392
wav_file_name : 14_18_3_1_k61.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)
Data index: 2231
wav_file_name : 14_34_2_1_k14.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)
Data index: 2527
wav_file_name : 4_36_3_1_k10.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)
Data index: 1920
wav_file_name : 6_14_3_9_k74.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)
Data index: 2480
wav_file_name : 6_34_1_7_k72.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)

wav_npdata.shape : (5, 128, 128, 3)


In [7]:
model = load_model(WEIGHT_PATH)
model.summary()


2022-03-06 22:57:59.513340: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-06 22:57:59.514495: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-06 22:57:59.654917: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3e:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-03-06 22:57:59.656445: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:b2:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-03-06 22:57:59.656467: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 63, 63, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 63, 63, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 63, 63, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

如果是一個資料才需要擴充維度

In [14]:

tf_wav_npdata = tf.expand_dims(wav_npdata, axis=0)
print(f"tf_wav_npdata.shape : {tf_wav_npdata.shape}" )

<dtype: 'float32'>
tf_wav_npdata.shape : (1, 128, 128, 3)


In [10]:
test = model.predict(wav_npdata)
print(f"predict value: {test}")

predict value: [[1.2029023e-09 1.0200034e-07 1.2183268e-09 9.9999988e-01]
 [3.9468215e-08 1.5189843e-07 3.4770227e-07 9.9999940e-01]
 [9.9999857e-01 3.4906512e-11 3.7098756e-17 1.4411860e-06]
 [6.4901409e-09 1.0000000e+00 7.8237902e-11 9.0660190e-10]
 [1.7113962e-04 4.5468977e-01 2.6290378e-04 5.4487616e-01]]


In [11]:
y_pred = np.argmax(test,axis=1)

print(y_pred)


[3 3 0 1 3]


In [ ]:
y_true = to_categorical(y_true,len(classes))
y_pred = to_categorical(y_pred,len(classes))

## test

In [ ]:
x = np.array([[[0,1],[2,3]],[[4,5],[6,7]]])
swapaxes

In [26]:
BATCH_SIZE = 128
tf_x =tf.data.Dataset.from_tensor_slices(
            (x_train, y_train)).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
print(tf_x)

<BatchDataset shapes: ((None, 128, 128, 3), (None, 4)), types: (tf.float32, tf.float32)>
